In [1]:
#importing openCV
import cv2

#importing numpy
import numpy as np

#importing pandas to read the CSV file containing our data
import pandas as pd

import datetime
import tensorflow as tf
import keras

#importing keras and sub-libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Conv2D, BatchNormalization, ELU
from tensorflow.keras import optimizers
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
def extract_plate(img):
    plate_img = img.copy()
    
    plate_cascade = cv2.CascadeClassifiers('indian_license_plate.xml')
    
    plate_rect = plate_cascade.detectMultiScale(plate_img,scaleFactor=1.3,minNeighbors=7)
    
    for (x,y,w,h) in plate_rect:
        a,b = (int(0.02*img.shape[0]),int(0.025*img.shape[1]))
        plate = plate_img[y+a:y+h-a,x+b:x+w-b,:]
        cv2.rectangle(plate_img,(x,y),(x+w,y+h),(51,51,255),3)
        
    return plate_img,plate

In [3]:
def segment_characters(img):
    img1 = cv2.resize(img,(333,75))
    img_gray = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    _,img_binary = cv2.threshold(img_gray,200,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_erode = cv2.erode(img_binary,(3,3))
    img_dilate = cv2.dilate(img_erode,(3,3))
    
    LP_WIDTH = img_dilate.shape[0]
    LP_HEIGHT = img_dilate.shape[1]
    
    img_dilate[0:3,:] = 255
    img_dilate[:,0:3] = 255
    img_dilate[72:75,:] = 255
    img_dilate[:,330:333] = 255
    
    dimensions = [LP_WIDTH/6,LP_WIDTH/2,LP_HEIGHT/10,2*LP_HEIGHT/3]
    
    char_list = find_contours(dimensions,img_dilate)
    
    return char_list

In [4]:
def find_contours(dimensions,img):
    cntrs,_ = cv2.findContours(img.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]
    
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]
    
    x_cntr_list = []
    target_contours = []
    img_res = []
    
    for contour in cntrs:
        intX,intY,intW,intH = cv2.boundingRect(contour)
        
        if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
            x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

            char_copy = np.zeros((44,24))
            #extracting each character using the enclosing rectangle's coordinates.
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            char = cv2.resize(char, (20, 40))

            # Make result formatted for classification: invert colors
            char = cv2.subtract(255, char)

            # Resize the image to 24x44 with black border
            char_copy[2:42, 2:22] = char
            char_copy[0:2, :] = 0
            char_copy[:, 0:2] = 0
            char_copy[42:44, :] = 0
            char_copy[:, 22:24] = 0

            img_res.append(char_copy) #List that stores the character's binary image (unsorted)
            
    indices = sorted(range(len(x_cntr_list)),key=lambda k: x_cntr_list[k])
    
    img_res_copy = []
    
    for idx in indices:
        img_res_copy.append(img_res[idx])
    
    img_res = np.array(img_res_copy)
    
    return img_res

In [5]:
model = Sequential()
model.add(Conv2D(filters=16,kernel_size=(3,3),input_shape=(28,28,1),activation='relu'))
model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=48,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=36,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.00001),metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 48)        13872     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 48)        0         
_________________________________________________________________
dropout (Dropout)            (None, 11, 11, 48)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5808)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               7

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.05,height_shift_range=0.05)

train_generator = train_datagen.flow_from_directory('data/train',target_size=(28,28),batch_size=1,color_mode='grayscale',class_mode='categorical')
validation_generator = train_datagen.flow_from_directory('data/val',target_size=(28,28),batch_size=1,color_mode='grayscale',class_mode='categorical')

Found 864 images belonging to 36 classes.
Found 216 images belonging to 36 classes.


In [8]:
class stop_training_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if logs.get('val_acc') > 0.99:
            self.model.stop_training = True

log_dir = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

batch_size = 1
callbacks = [tensorboard_callback, stop_training_callback()]
model.fit_generator(train_generator,validation_data=validation_generator,epochs=100,callbacks=callbacks)

Epoch 1/100
864/864 [==============================] - 19s 22ms/step - loss: 3.5534 - acc: 0.0405 - val_loss: 3.4872 - val_acc: 0.1157
Epoch 2/100
864/864 [==============================] - 18s 21ms/step - loss: 3.3986 - acc: 0.1701 - val_loss: 3.2058 - val_acc: 0.4167
Epoch 3/100
864/864 [==============================] - 18s 21ms/step - loss: 2.9344 - acc: 0.3611 - val_loss: 2.5096 - val_acc: 0.5880
Epoch 4/100
864/864 [==============================] - 18s 21ms/step - loss: 2.0336 - acc: 0.5625 - val_loss: 1.5387 - val_acc: 0.7130
Epoch 5/100
864/864 [==============================] - 18s 21ms/step - loss: 1.2894 - acc: 0.6806 - val_loss: 0.9642 - val_acc: 0.8056
Epoch 6/100
864/864 [==============================] - 18s 21ms/step - loss: 0.9895 - acc: 0.7222 - val_loss: 0.7216 - val_acc: 0.8380
Epoch 7/100
864/864 [==============================] - 18s 21ms/step - loss: 0.7729 - acc: 0.7708 - val_loss: 0.6249 - val_acc: 0.8380
Epoch 8/100
864/864 [==============================] - 